In [ ]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy.random as npr

import matplotlib
import matplotlib.animation 
import matplotlib.pyplot as plt
matplotlib.rcParams["animation.embed_limit"] = 1024

import skimage
import skimage.io as sio
import skimage.transform
import fracatal

from fracatal.functional_pt.convolve import ft_convolve
from fracatal.functional_pt.compose import make_gaussian, \
        make_mixed_gaussian, \
        make_kernel_field, \
        make_update_function, \
        make_update_step, \
        make_make_kernel_function

from fracatal.functional_pt.metrics import compute_entropy, \
        compute_frequency_ratio, \
        compute_frequency_entropy
        
from fracatal.scripts import v_stability_sweep, stability_sweep     

import IPython

In [ ]:
# pattern params (physical and simulation)
params = {"orbium_unicaudatus": {"mu_k": [0.5], \
                                 "sigma_k": [0.15],\
                                 "amplitudes_k": [1.0],\
                                 "mu_g": 0.15, \
                                 "sigma_g": 0.017,\
                                 "dt": 0.1,\
                                 "kr": 13,\
                                 "notes": "from https://chakazul.github.io/Lenia/JavaScript/Lenia.html"\
                                },\
        "orbium_unicaudatus_0": {"mu_k": [0.5], \
                                 "sigma_k": [0.15],\
                                 "amplitudes_k": [1.0],\
                                 "mu_g": 0.15, \
                                 "sigma_g": 0.016,\
                                 "dt": 0.1,\
                                 "kr": 13,\
                                 "notes": "from Chan 2019 in Complex Systems 28, e.g. Fig. 5"\
                                },\
         "orbium_bicaudatus": {"mu_k": [0.5], \
                                 "sigma_k": [0.15], \
                                 "amplitudes_k": [1.0],\
                                 "mu_g": 0.15, \
                                 "sigma_g": 0.014,\
                                 "dt": 0.1,\
                                 "kr": 13,\
                                 "notes": "from https://chakazul.github.io/Lenia/JavaScript/Lenia.html"\
                                },\
         "orbium_bicaudatus_ignis": {"mu_k": [0.5], \
                                 "sigma_k": [0.15], \
                                 "amplitudes_k": [1.0],\
                                 "mu_g": 0.12, \
                                 "sigma_g": 0.0113,\
                                 "dt": 0.1,\
                                 "kr": 13,\
                                 "notes": "from https://chakazul.github.io/Lenia/JavaScript/Lenia.html"\
                                },\
         "asymdrop": {"mu_k": [0.5], \
                                 "sigma_k": [0.15], \
                                 "amplitudes_k": [1.0],\
                                 "mu_g": 0.12, \
                                 "sigma_g": 0.005,\
                                 "dt": 0.2,\
                                 "kr": 13,\
                                  "notes": "from Q. Davis 2024 in ALife Proc."\
                                 }
         }

In [ ]:
"""
animation functions
"""

def get_fig(grid, my_cmap="jet", invert=False, annotate_fig=True):
    
    global subplot_0
    
    fig, ax = plt.subplots(1,1, facecolor="black", figsize=(6,6))
    
    if grid.shape[1] == 3:
        subplot_0 = ax.imshow(grid.squeeze().permute(1,2,0))
    else:
        if invert:
            display_grid = 1.0 - plt.get_cmap(my_cmap)(grid.squeeze())
        else:
            display_grid = plt.get_cmap(my_cmap)(grid.squeeze())
        subplot_0 = ax.imshow(display_grid, cmap=my_cmap)

    ax.set_xticklabels("")
    ax.set_yticklabels("")
    
    my_dt = dts[0]
    my_kr = krs[0]
    
    corner_x = .08 #/ (grid_dim*6)
    corner_y = .08# / (grid_dim*6)
    step_y = .08#/ (grid_dim*6)
    
    if annotate_fig:
        fig.text(corner_x, corner_y, "$\Delta t =$ " + f"{my_dt:.2e}", fontsize=22, color="w")
        fig.text(corner_x, corner_y + 1 * step_y, "$1/k_r =$ " + f"1/{my_kr:.1f} = " + " $\Delta r = $ " +  f" {1/my_kr:.2e}",\
                fontsize=22, color="w")
        fig.text(corner_x, corner_y + 2 * step_y, "$\mu_g =$ " + f"{mu_g:.2e}", fontsize=22, color="w")
        fig.text(corner_x, corner_y + 3 * step_y, "$\sigma_g =$ " + f"{sigma_g:.2e}", fontsize=22, color="w")
        
    
    def update_frame(ii):
        
        global grid
        global dts
        global krs
    
        if grid.shape[1] == 3:
            subplot_0.set_array(grid.squeeze().permute(1,2,0))
        else:
            subplot_0.set_array(grid.squeeze())
    
        my_dt = dts[ii]
        my_kr = krs[ii]
        
        kernel = make_kernel(my_kr)    
        update_step = make_update_step(my_update, kernel, my_dt, clipping_function=clipping_fn, mode=mode)\
    
        corner_x = 8 / (grid_dim*6)
        corner_y = 16 / (grid_dim*6)
        step_y = 16/ (grid_dim*6)
        
        if annotate_fig:
            fig.texts[0].set_text("$\Delta t =$ " + f"{my_dt:.2e}")
            fig.texts[1].set_text("$1/k_r =$ " + f"1/{my_kr:.1f} = " + " $\Delta r = $ " +  f" {1/my_kr:.2e}")
            fig.texts[2].set_text("$\mu_g =$ " + f"{mu_g:.2e}")
            fig.texts[3].set_text("$\sigma_g =$ " + f"{sigma_g:.2e}")
            
        plt.tight_layout()
        
        grid = update_step(grid)

    return fig, ax, update_frame
        

# Gliders in Lenia approximate their mathematical "true selves". 

### Lenia's mathematical description

$$
A(x,t_0+\Delta t) = [A(x,t_0) + \Delta t * G(K \circledast A(x,t_0))]_0^1
$$
<div align="center">
Equation 1. $*$ is used for scalar multiplication and $\circledast$ is convolution. $|\cdot|_0^1$ denotes clipping to a range between 0 and 1.
</div>
<br>


You may notice striking similarity between the Lenia update in Equation 1 and Euler's method and iterative optimization methods (_i.e._ stochastic gradient descent).


### pseudocode implementation

```
# given
# a: cell states
# k: neighborhood kernel
# dt: step size
# grow: growth function
# conv: convolution function

convolved_a = conv(k, a)
growth = grow(convolved_a)

# update a
a = clip(a + dt * growth, 0., 1.)
```


### single line version

```
a = clip(a + dt * grow(conv(k,a)), 0., 1.)
```


In [ ]:
pattern_name = "orbium_unicaudatus"
my_cmap = "magma"
save_vid = True
tag = "july11"

pattern = np.load(os.path.join("..", "patterns", f"{pattern_name}.npy"))
pattern_params = params[pattern_name]

grid_dim = 64
kr = pattern_params["kr"]
k0 = 1 * kr
amplitudes = pattern_params["amplitudes_k"]
means_k = pattern_params["mu_k"]
standard_deviations_k = pattern_params["sigma_k"]
mu_g = pattern_params["mu_g"]
sigma_g = 0.0203 #pattern_params["sigma_g"]

start_dt = 0.05
start_kr = 1 * kr

end_dt = start_dt - 1e-7
end_kr = start_kr + 1e-7
base_steps = 512
number_frames = base_steps

ddt = (end_dt-start_dt) / (number_frames-1)
dkr = (end_kr-start_kr) / (number_frames-1)

dts = np.arange(start_dt, end_dt+ddt, ddt)
krs = np.arange(start_kr, end_kr+dkr, dkr) 

if pattern_name == "asymdrop":
    mode = 1
    # identity (no clipping)
    clipping_fn = lambda x: x
else:
    mode = 0
    clipping_fn = lambda x: np.clip(x, 0, 1.0)


make_kernel = make_make_kernel_function(amplitudes, means_k, standard_deviations_k, dim=grid_dim-6)
my_update = make_update_function(mu_g, sigma_g, mode=mode)

half_dim = grid_dim // 2 - max(pattern.shape)
grid = torch.zeros(1,1, grid_dim, grid_dim)

grid[:,:,half_dim:half_dim+pattern.shape[-2], half_dim:half_dim+pattern.shape[-1]] = torch.tensor(pattern)

annotate_fig = True


fig, ax, update_frame = get_fig(grid, my_cmap = my_cmap, annotate_fig=annotate_fig)
plt.close()

my_interval = 30

sio.imsave(f"vid0a_thumbnail.png", np.uint8(255*plt.get_cmap(my_cmap)(grid.squeeze())[...,:3]))
#.save(f"{pattern_name}_{tag}.mp4")     
matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval)\
        .save(f"vid0a_{pattern_name}_{tag}.mp4") if save_vid \
        else IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval).to_jshtml())

In [ ]:
pattern_name = "orbium_unicaudatus"
my_cmap = "magma"
save_vid = False
tag = "july10"

pattern = np.load(os.path.join("..", "patterns", f"{pattern_name}.npy"))
pattern_params = params[pattern_name]

grid_dim = 384
kr = pattern_params["kr"]
k0 = 1 * kr
amplitudes = pattern_params["amplitudes_k"]
means_k = pattern_params["mu_k"]
standard_deviations_k = pattern_params["sigma_k"]
mu_g = pattern_params["mu_g"]
sigma_g = 0.0203 #pattern_params["sigma_g"]

start_dt = 0.1
start_kr = 1 * kr

end_dt = 0.01
end_kr = 130
base_steps = 1024
number_frames = base_steps

ddt = (end_dt-start_dt) / (number_frames-1)
dkr = (end_kr-start_kr) / (number_frames-1)

dts = np.arange(start_dt, end_dt+ddt, ddt)
krs = np.arange(start_kr, end_kr+dkr, dkr) 

if pattern_name == "asymdrop":
    mode = 1
    # identity (no clipping)
    clipping_fn = lambda x: x
else:
    mode = 0
    clipping_fn = lambda x: np.clip(x, 0, 1.0)


make_kernel = make_make_kernel_function(amplitudes, means_k, standard_deviations_k, dim=grid_dim-6)
my_update = make_update_function(mu_g, sigma_g, mode=mode)

half_dim = grid_dim // 2 - max(pattern.shape)
grid = torch.zeros(1,1, grid_dim, grid_dim)

grid[:,:,half_dim:half_dim+pattern.shape[-2], half_dim:half_dim+pattern.shape[-1]] = torch.tensor(pattern)

annotate_fig = True


fig, ax, update_frame = get_fig(grid, my_cmap = my_cmap, annotate_fig=annotate_fig)
plt.close()

my_interval = 30

sio.imsave(f"vid1a_thumbnail.png", np.uint8(255*plt.get_cmap(my_cmap)(grid.squeeze())[...,:3]))
matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval)\
        .save(f"vid1a_{pattern_name}_{tag}.mp4") if save_vid \
        else IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval).to_jshtml())

In [ ]:
start_dt = end_dt * 1.
start_kr = 1 * end_kr

end_dt = start_dt + 1e-6
end_kr = start_kr + 1e-6
number_frames = base_steps * 4

ddt = (end_dt-start_dt) / (number_frames-1)
dkr = (end_kr-start_kr) / (number_frames-1)

dts = np.arange(start_dt, end_dt+ddt, ddt)
krs = np.arange(start_kr, end_kr+dkr, dkr) 

my_interval = 6
speedup = 1

fig, ax, update_frame = get_fig(grid, my_cmap = my_cmap)
plt.close()

sio.imsave(f"vid1b_thumbnail.png", np.uint8(255*plt.get_cmap(my_cmap)(grid.squeeze())[...,:3]))
matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval)\
        .save(f"vid1b_{pattern_name}_{tag}.mp4") if save_vid \
        else IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval).to_jshtml())

# Or do they? 

In [ ]:
# copy grid

grid_0 = 1.0 * grid
end_dt_0 = 1.0 * end_dt
end_kr_0 = 1.0 * end_kr


In [ ]:
start_dt = end_dt_0 * 1.
start_kr = 1 * end_kr_0

end_dt = 0.00001
end_kr = start_kr + 1e-6
number_frames = base_steps * 4

ddt = (end_dt-start_dt) / (number_frames-1)
dkr = (end_kr-start_kr) / (number_frames-1)

dts = np.arange(start_dt, end_dt+ddt, ddt)
krs = np.arange(start_kr, end_kr+dkr, dkr) 


my_interval = 6
speedup = 1

fig, ax, update_frame = get_fig(grid, my_cmap = my_cmap)
plt.close()


sio.imsave(f"vid1c_thumbnail.png", np.uint8(255*plt.get_cmap(my_cmap)(grid.squeeze())[...,:3]))
matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval)\
        .save(f"vid1c_{pattern_name}_{tag}.mp4") if save_vid \
        else IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval).to_jshtml())

# Inspiration: Kojima and Ikegami 2023 


# Does non-Platonic persistence depend on the clipping function

# Asymptotic Lenia (ALenia)

### Lenia's mathematical description

$$
A(x,t_0+\Delta t)= A(x,t_0) + \Delta t * \left(A(x,t_0) - T(K \circledast A(x,t_0)\right)
$$
<div align="center">
Equation 2. Note that ALenia does not use a clipping function. 
</div>
<br>


### pseudocode implementation

```
# given
# a: cell states
# k: neighborhood kernel
# dt: step size
# calc_target: growth function
# conv: convolution function

convolved_a = conv(k, a)
target = calc_target(convolved_a)

# update a
a = a + dt * (a-target)
```


### single line version

```
a = a + dt * (a - calc_target(conv(k,a)))
```


In [ ]:
pattern_name = "asymdrop"
my_cmap = "magma"
tag = ""
my_seed = 13
my_seed = 42

np.random.seed(my_seed)
torch.manual_seed(my_seed)

pattern = np.load(os.path.join("..", "patterns", f"{pattern_name}.npy"))
pattern_params = params[pattern_name]

grid_dim = 384
kr = pattern_params["kr"]
k0 = 1 * kr
amplitudes = pattern_params["amplitudes_k"]
means_k = pattern_params["mu_k"]
standard_deviations_k = pattern_params["sigma_k"]

start_dt = 0.2
start_kr = 1 * kr

end_dt = start_dt - 1e-7
end_kr = start_kr + 1e-7

base_steps = 1024
number_frames = base_steps

ddt = (end_dt-start_dt) / (number_frames-1)
dkr = (end_kr-start_kr) / (number_frames-1)

dts = np.arange(start_dt, end_dt+ddt, ddt)
krs = np.arange(start_kr, end_kr+dkr, dkr) 

if pattern_name == "asymdrop":
    mode = 1
    # identity (no clipping)
    clipping_fn = lambda x: x
else:
    mode = 0
    clipping_fn = lambda x: np.clip(x, 0, 1.0)

my_iteration = 0

if save_vid:
    for mu_g, sigma_g in ((0.12, 0.015),\
                          (0.12, 0.001),\
                          (0.12, 0.0075),\
                          (0.12, 0.005)\
                         ):
        
        np.random.seed(my_seed)
        torch.manual_seed(my_seed)
        
        make_kernel = make_make_kernel_function(amplitudes, means_k, standard_deviations_k, dim=grid_dim-6)
        my_update = make_update_function(mu_g, sigma_g, mode=mode)
        
        kernel = make_kernel(krs[0])
        my_dt = 1. * start_dt
        update_step = make_update_step(my_update, kernel, my_dt, clipping_function=clipping_fn, mode=mode)\
        
        # populate a grid with random uniform patches
        grid_stride = 66
        patch_dim = 41
        
        grid = torch.zeros(1,1,grid_dim, grid_dim)
        
        for hh in range(grid_stride, grid_dim, grid_stride):
            for ww in range(0, grid_dim, grid_stride):
        
                grid[:,:,hh:hh+patch_dim, ww:ww+patch_dim] = torch.rand(1,1, patch_dim, patch_dim)
    
        if save_vid:    
            annotate_fig = False
            fig, ax,_ = get_fig(grid, my_cmap = my_cmap, annotate_fig=annotate_fig)    
            fig.suptitle("randum uniform U(0,1) patch initialization")
            plt.savefig("uniform_patches_alenia.png") 
            plt.close()
        
        grow_steps = 96
        my_interval = 30
        
        if(0):
            for my_step in range(grow_steps):
                grid = update_step(grid)
        
        annotate_fig = True
        fig, ax, update_frame  = get_fig(grid, my_cmap = my_cmap, annotate_fig=annotate_fig)    
        plt.close()
        
        matplotlib.animation.FuncAnimation(fig, update_frame, frames=grow_steps, interval=my_interval)\
                .save(f"vid2a_alenia_initial{my_iteration}_{tag}.mp4")
        
        sio.imsave(f"vid2a_{my_iteration}_thumbnail.png", np.uint8(255*plt.get_cmap(my_cmap)(grid.squeeze())[...,:3]))
        my_iteration += 1

In [ ]:
# This cell is for visualizing in notebook
# the IPython.display.HTML(...).to_jshtml() line does not work for in-notebook visualization from inside a loop

pattern_name = "asymdrop"
my_cmap = "magma"
tag = ""
my_seed = 13
my_seed = 42

np.random.seed(my_seed)
torch.manual_seed(my_seed)

pattern = np.load(os.path.join("..", "patterns", f"{pattern_name}.npy"))
pattern_params = params[pattern_name]

grid_dim = 384
kr = pattern_params["kr"]
k0 = 1 * kr
amplitudes = pattern_params["amplitudes_k"]
means_k = pattern_params["mu_k"]
standard_deviations_k = pattern_params["sigma_k"]
mu_g = pattern_params["mu_g"]
sigma_g = pattern_params["sigma_g"]

start_dt = 0.2
start_kr = 1 * kr

end_dt = start_dt - 1e-7
end_kr = start_kr + 1e-7

base_steps = 1024
number_frames = base_steps

ddt = (end_dt-start_dt) / (number_frames-1)
dkr = (end_kr-start_kr) / (number_frames-1)

dts = np.arange(start_dt, end_dt+ddt, ddt)
krs = np.arange(start_kr, end_kr+dkr, dkr) 

if pattern_name == "asymdrop":
    mode = 1
    # identity (no clipping)
    clipping_fn = lambda x: x
else:
    mode = 0
    clipping_fn = lambda x: np.clip(x, 0, 1.0)

make_kernel = make_make_kernel_function(amplitudes, means_k, standard_deviations_k, dim=grid_dim-6)
my_update = make_update_function(mu_g, sigma_g, mode=mode)

kernel = make_kernel(krs[0])
my_dt = 1. * start_dt
update_step = make_update_step(my_update, kernel, my_dt, clipping_function=clipping_fn, mode=mode)\

# populate a grid with random uniform patches
grid_stride = 66
patch_dim = 41

grid = torch.zeros(1,1,grid_dim, grid_dim)

for hh in range(grid_stride, grid_dim, grid_stride):
    for ww in range(0, grid_dim, grid_stride):

        grid[:,:,hh:hh+patch_dim, ww:ww+patch_dim] = torch.rand(1,1, patch_dim, patch_dim)
   
if save_vid:
        
    annotate_fig = False
    fig, ax, update_frame = get_fig(grid, my_cmap = my_cmap, annotate_fig=annotate_fig) 
    fig.suptitle("randum uniform U(0,1) patch initialization")
    plt.savefig("uniform_patches_alenia.png")


grow_steps = 96
my_interval = 30

if(0):
    for my_step in range(grow_steps):
        grid = update_step(grid)

annotate_fig = False

fig, ax, update_frame  = get_fig(grid, my_cmap = my_cmap, annotate_fig=annotate_fig)    
plt.close()

sio.imsave(f"vid2a_thumbnail.png", np.uint8(255*plt.get_cmap(my_cmap)(grid.squeeze())[...,:3]))
matplotlib.animation.FuncAnimation(fig, update_frame, frames=grow_steps, interval=my_interval)\
        .save(f"vid2a_alenia_initial{pattern_name}_{tag}.mp4") if save_vid \
        else IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_frame, frames=grow_steps, interval=my_interval).to_jshtml())


In [ ]:

grid_1 = 1 * grid

fig1, ax1 = plt.subplots(1,2,figsize=(9,6)) 

ax1[0].imshow(grid_1.squeeze(), cmap=my_cmap)

ax1[1].imshow(grid_1.squeeze(), cmap=my_cmap)

my_alpha = 0.4
rectangles = []
rectangles.append(matplotlib.patches.Rectangle((0, 0), 320, 167, color="g", alpha=my_alpha))
rectangles.append(matplotlib.patches.Rectangle((0, 0), 40, grid_dim, color="g", alpha=my_alpha))
rectangles.append(matplotlib.patches.Rectangle((350, 0), grid_dim-350, 200, color="g", alpha=my_alpha))
rectangles.append(matplotlib.patches.Rectangle((335, 200), grid_dim-335, 60, color="g", alpha=my_alpha))
rectangles.append(matplotlib.patches.Rectangle((0, 125), grid_dim, 42, color="g", alpha=my_alpha))

for rectangle in rectangles:
    ax1[1].add_patch(rectangle)

for ii in range(2):
    ax1[ii].set_xticklabels("")
    ax1[ii].set_yticklabels("")

ax1[0].set_xlabel(f"grid after {grow_steps} steps", fontsize=18)
ax1[1].set_xlabel(f"cropping to gliders\n(green areas set to 0)", fontsize=18)
fig1.suptitle("Selecting ALenia gliders at \n$\mu_T =$ " + f"{mu_g} " + "and $\sigma_T = $ " + f"{sigma_g}", fontsize=24)

plt.tight_layout()
if save_vid:
    plt.savefig(f"{tag}_alenia_crop.png")
plt.show()

grid_1[:,:,:167,:320] *= 0
grid_1[:,:,:,:40] *= 0
grid_1[:,:,:200,350:] *= 0
grid_1[:,:,200:260,335:] *= 0

grid_1[:,:,125:167,:] *= 0
grid = grid_1*1

annotate_fig = False
fig, ax, update_frame = get_fig(grid, my_cmap = my_cmap, annotate_fig=annotate_fig) 
plt.close()

sio.imsave(f"vid2b_thumbnail.png", np.uint8(255*plt.get_cmap(my_cmap)(grid.squeeze())[...,:3]))
matplotlib.animation.FuncAnimation(fig, update_frame, frames=grow_steps*8, interval=my_interval)\
        .save(f"vid2b_alenia_cropped_{pattern_name}_{tag}.mp4") if save_vid \
        else IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_frame, frames=grow_steps*8, interval=my_interval).to_jshtml())

In [ ]:


start_dt = 0.25
start_kr = 1 * kr

end_dt = 0.1
end_kr = 130
base_steps = 1024
number_frames = base_steps

ddt = (end_dt-start_dt) / (number_frames-1)
dkr = (end_kr-start_kr) / (number_frames-1)

dts = np.arange(start_dt, end_dt+ddt, ddt)
krs = np.arange(start_kr, end_kr+dkr, dkr) 

if pattern_name == "asymdrop":
    mode = 1
    # identity (no clipping)
    clipping_fn = lambda x: x
else:
    mode = 0
    clipping_fn = lambda x: np.clip(x, 0, 1.0)


make_kernel = make_make_kernel_function(amplitudes, means_k, standard_deviations_k, dim=grid_dim-6)
my_update = make_update_function(mu_g, sigma_g, mode=mode)

half_dim = grid_dim // 2 - max(pattern.shape)
grid = torch.zeros(1,1, grid_dim, grid_dim)

grid[:,:,half_dim:half_dim+pattern.shape[-2], half_dim:half_dim+pattern.shape[-1]] = torch.tensor(pattern)

annotate_fig = True
fig, ax, update_frame = get_fig(grid, my_cmap = my_cmap, annotate_fig=annotate_fig)
plt.close()

my_interval = 30

sio.imsave(f"vid3_thumbnail.png", np.uint8(255*plt.get_cmap(my_cmap)(grid.squeeze())[...,:3]))
matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval)\
        .save(f"vid3_alenia_nonplatonic_{pattern_name}_{tag}.mp4") if save_vid \
        else IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval).to_jshtml())

# Miscellaneous figures

In [ ]:
save_figs = False
fig, ax = plt.subplots(1,2, figsize=(9,3), gridspec_kw = {"width_ratios": [0.25, 0.75]})

kr = 13
x = np.arange(0, 1.0, 0.001)
amplitudes = [1.0]
means_k = [0.5]
standard_deviations_k = [0.15]
grid_dim = 75

mu_g = 0.15
sigma_g = 0.017
mode = 0

my_update = make_update_function(mu_g, sigma_g, mode=mode)

make_kernel = make_make_kernel_function(amplitudes, means_k, standard_deviations_k, dim=grid_dim-6)
kernel = make_kernel(kr)

pattern_name = "orbium_unicaudatus"
pattern = np.load(os.path.join("..", "patterns", f"{pattern_name}.npy"))
grid = torch.zeros(1,1,grid_dim, grid_dim)
grid[:,:,32:32+pattern.shape[-2], 32:32+pattern.shape[-1]] = torch.tensor(pattern)
neighborhood = ft_convolve(grid, kernel)
growth = my_update(neighborhood)

ax[0].imshow(kernel.squeeze(), cmap="magma")
ax[1].plot(x, my_update(x), color=plt.get_cmap("magma")(48), lw=4)
ax[1].set_title("Growth function $G(n) = 2*(e^{-(\mu-n)^2/2 \sigma^2})-1$")
ax[0].set_title("Neighborhood kernel")
ax[0].set_xticklabels("")
ax[0].set_yticklabels("")

if (save_figs):
    plt.savefig("kernel_growth.png")

plt.tight_layout()
plt.show()
fig1, ax1 = plt.subplots(1,3, figsize=(9,2.6))

ax1_b = plt.twinx(ax1[0])
ax1[0].imshow(neighborhood.squeeze(), cmap="magma")
ax1[0].set_title("cell neighborhoods $K\circledast A(x,t)$")
ax1[0].set_xticklabels("")
ax1[0].set_yticklabels("")
ax1_b.set_yticklabels("")
ax1_b.set_ylabel("     $=$    ", fontsize=32, rotation=0)

ax1[1].imshow(grid.squeeze(), cmap="magma")
ax1[1].set_title("cell states $A(x,t)$")
ax1[1].set_xticklabels("")
ax1[1].set_yticklabels("")

ax1_d = plt.twinx(ax1[2])
ax1[2].imshow(kernel.squeeze(), cmap="magma")
ax1[2].set_title("neighborhood kernel $K$")
ax1[2].set_xticklabels("")
ax1[2].set_yticklabels("")
ax1_d.set_yticklabels("")

fig1.text(0.3,.24,"$($", fontsize=128, rotation=0)

fig1.text(0.63,.3,"$\circledast$", fontsize=72, rotation=0)
fig1.text(1.0,.24,"$)$", fontsize=128, rotation=0)

plt.tight_layout()

if (save_figs):
    plt.savefig("neighborhood_equals.png")
                 
fig1, ax1 = plt.subplots(1,1, figsize=(3,3))

ax1.imshow(growth.squeeze(), cmap="magma")
ax1.set_title("growth $G(K\circledast A(x,t))$")

ax1.set_xticklabels("")
ax1.set_yticklabels("")
ax1_b.set_yticklabels("")

plt.tight_layout()
if (save_figs):
    plt.savefig("growth.png")

fig2, ax2 = plt.subplots(1,1, figsize=(3,3))
ax2.imshow(neighborhood.squeeze(), cmap="magma")
ax2.set_title("cell neighborhoods $K\circledast A(x,t)$")
ax2.set_xticklabels("")
ax2.set_yticklabels("")
fig2.text(-0.7,.35,"$=G($", fontsize=64, rotation=0)
fig2.text(1,.35,"$)$", fontsize=64, rotation=0)

plt.tight_layout()

if (save_figs):
    fig2.savefig("equals_gneighborhood.png")
    
dt= 0.1
new_grid = grid + dt * growth

fig3, ax3 =  plt.subplots(1,1, figsize=(3,3))


ax3.imshow(new_grid.squeeze(), cmap="magma")
ax3.set_title("cell states $A(x,t+\Delta t)$")

ax3.set_xticklabels("")
ax3.set_yticklabels("")
plt.tight_layout()

if (save_figs):
    plt.savefig("new_grid.png")

fig1, ax1 = plt.subplots(1,3, figsize=(9,2.3))

ax1_b = plt.twinx(ax1[0])
ax1[0].imshow(new_grid.squeeze(), cmap="magma")
ax1[0].set_title("cell states $A(x,t+\Delta t)$")
ax1[0].set_xticklabels("")
ax1[0].set_yticklabels("")
ax1_b.set_yticklabels("")
ax1_b.set_ylabel("       $=$", fontsize=32, rotation=0)

ax1_c = plt.twinx(ax1[1])
ax1[1].imshow(grid.squeeze(), cmap="magma")
ax1[1].set_title("cell states $A(x,t)$")
ax1[1].set_xticklabels("")
ax1[1].set_yticklabels("")
ax1_c.set_yticklabels("")
ax1_c.set_ylabel("        $+ \Delta t *$", fontsize=32, rotation=0)

ax1_d = plt.twinx(ax1[2])
ax1[2].imshow(growth.squeeze(), cmap="magma")
ax1[2].set_title("growth $G(K\circledast A(x,t))$")
ax1[2].set_xticklabels("")
ax1[2].set_yticklabels("")
ax1_d.set_yticklabels("")


plt.tight_layout()

if (save_figs):
    plt.savefig("update.png")
                 
plt.show()